# Import von WIAG-Daten in FactGrid
Formatiere CSV-Daten aus WIAG als V1-Statements für QuickSteps  
siehe [Einführung in QuickStatements](https://www.wikidata.org/wiki/Help:QuickStatements)
und die [Properties des FactGrid](https://database.factgrid.de/wiki/FactGrid:Directory_of_Properties)

Änderungen

- *2023-12-11* Verwende die Gruppe des höchsten erreichten Amtes in der englischen Beschreibung und für P165

Inhalt:

- [neue Objekte in FactGrid anlegen](#neue-Objekte-in-FactGrid-anlegen)

In [1]:
using CSV, DataFrames, Dates, URIs

In [2]:
ENV["COLUMNS"] = 120

120

In [3]:
input_path = "C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid"

"C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid"

In [4]:
output_path = input_path

"C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid"

## neue Objekte in FactGrid anlegen
Es wird davon ausgegangen, dass die betreffenden Personen noch nicht in FactGrid vorhanden sind. Die Spalte "FactGrid_ID" sollte also für alle Einträge leer sein.

Daten aus WIAG einlesen: Quelle Export "CSV Personendaten"

In [5]:
input_file = "WIAG-can_2023-12-13_ii.csv"
input_path_file = joinpath(input_path, input_file)

"C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid\\WIAG-can_2023-12-13_ii.csv"

In [6]:
df_person_all = CSV.read(input_path_file, DataFrame);
nrow(df_person_all)

15144

Betrachte nur Neu-Zugänge (keine FactGrid-ID)

In [7]:
df_person = subset(df_person_all, :FactGrid_ID => ByRow(ismissing));
nrow(df_person)

13878

Das deutsche Beschreibungsfeld enthält Lebensdaten mit der Zusammenfassung der Amtsdaten

In [8]:
join_komma(a, b) = join((a, b), ", ")
transform!(df_person, [:biographical_dates, :summary_offices] => ByRow(join_komma) => :description_de);

Das FactGrid arbeitet für Wikipedia nicht mit URL-codierten Adressen.
*2023-12-13* WIAG gibt direkt 

In [9]:
unescape_not_mg(s) = ismissing(s) ? s : URIs.unescapeuri(s)

unescape_not_mg (generic function with 1 method)

In [41]:
transform!(df_person, :Wikipedia => ByRow(unescape_not_mg) => :Wikipedia);

In [42]:
df_person[11:19, [:givenname, :prefix, :familyname, :Wikipedia]]

Row,givenname,prefix,familyname,Wikipedia
,String,String31?,String?,String?
1,Peter,missing,Fride,missing
2,Heinrich,missing,Elchleben,missing
3,Wilhelm,missing,Schenk von Limpurg,missing
4,Gebhard,von,Bulach,missing
5,Johannes,missing,Kautsch,missing
6,Johannes Wernher,von,Flachslanden,missing
7,Berthold,von,Henneberg,missing
8,Johann,missing,Pfalzgraf bei Rhein,Johann_von_Pfalz-Mosbach
9,Albrecht,von,Bayern,Albrecht_IV._(Bayern)


Benenne die Spalten um entsprechend den Konventionen des FactGrid

In [43]:
columns = [
    :displayname => :Lde,
    :description_de => :Dde,
    :date_of_birth => :P77,
    :date_of_death => :P38,
    :carrer => :P165,
    :GND_ID => :P76,
    :GSN => :P472,
    :id => :P601,
    :Wikidata_ID => :Swikidatawiki,
    :Wikipedia => :Sdewiki
];

In [44]:
rename!(df_person, columns);

Kopiere das Label in Deutsch für die anderen Sprachen

In [45]:
df_person.Len = df_person.Lde;
df_person.Lfr = df_person.Lde;
df_person.Les = df_person.Lde;

Erstelle eine Beschreibung für Englisch aus Lebensdaten und der Gruppe des höchsten Amtes

In [46]:
join_komma(a, b) = join([a, b], ", ")

join_komma (generic function with 1 method)

In [47]:
transform!(df_person, [:biographical_dates, :carrer_en] => ByRow(join_komma) => :Den);

In [48]:
df_person[11:19, [:givenname, :prefix, :familyname, :Len, :Den]]

Row,givenname,prefix,familyname,Len,Den
,String,String31?,String?,String,String
1,Peter,missing,Fride,Peter Fride,"~ 1417, Member of a collegiate church"
2,Heinrich,missing,Elchleben,Heinrich Elchleben,"+ kurz vor 1446, Cleric of the Cathedral chapter"
3,Wilhelm,missing,Schenk von Limpurg,Wilhelm Schenk von Limpurg,"* 1410, + 1475, Head of an (Arch)diocese"
4,Gebhard,von,Bulach,Gebhard von Bulach,"+ 1465, Cleric of the Cathedral chapter"
5,Johannes,missing,Kautsch,Johannes Kautsch,"+ 1462, Cleric of the Cathedral chapter"
6,Johannes Wernher,von,Flachslanden,Johannes Wernher von Flachslanden,"+ 1481, Dignitary of a cathedral chapter"
7,Berthold,von,Henneberg,Berthold von Henneberg,"* 1443, + 1495, Cleric of the Cathedral chapter"
8,Johann,missing,Pfalzgraf bei Rhein,Johann Pfalzgraf bei Rhein,"* 1443, + 1486, Dignitary of a cathedral chapter"
9,Albrecht,von,Bayern,Albrecht von Bayern,"+ 1508, Cleric of the Cathedral chapter"


Füge Daten ein, die für alle Personen gleich sind:
Mensch, Teil der Germania Sacra Forschungsdaten, männlich

In [49]:
insertcols!(df_person, 
    :P2 => "Q7",
    :P131 => "Q153178",
    :P154 => "Q18"
);    

Definiere Umwandlungsfunktionen  
*offen:* Datumsangaben sind noch zu besprechen, wegen der Verarbeitung von unscharften Zeitangaben.

In [50]:
fqs_string(s) = "\"" * string(s) * "\""

fqs_string (generic function with 1 method)

In [51]:
function create(out, row, col_list) 
    println(out, "CREATE")    
    for col in col_list
        println(out, "LAST\t", string(col), "\t", fqs_string(row[col]))        
    end
end

create (generic function with 1 method)

In [52]:
"""
    set_property_list (out, row, object, col_list, fmt_list)

    

# Arguments
- `out::IOStream`: output stream
- `row::DataFrameRow`: input data
- `object`: FactGrid identifier or "LAST"
- `col_list`: list of colums where data should be read from
- `fmr_list`: list of formatting functions (one for each column)
"""
function set_property_list(out, row, object, col_list, fmt_list)
    for (col, fmt) in zip(col_list, fmt_list)
        if (!ismissing(row[col]))
            println(out, object, "\t", string(col), "\t", fmt(row[col]))
        end
    end
end

set_property_list

Gib ausgewählte Elemente aus `df_person` aus. Falls es schon eine Datei mit gleichem Namen im angegebenen Verzeichnis gibt, wird die Datei überschrieben.

In [53]:
date_key = Dates.format(now(), "yyyy-mm-dd")

"2023-12-11"

In [54]:
output_file = "Insert_WIAG-can_" * date_key * "_FQ.csv"
output_path_file = joinpath(output_path, output_file)

"C:\\Users\\georg\\Documents\\projekte-doc\\Forschungsdaten-GS\\data\\FactGrid\\Insert_WIAG-can_2023-12-11_FQ.csv"

In [55]:
open(output_path_file, "w") do out_stream
    for row in eachrow(df_person)
        create(out_stream, row, [:Lde, :Len, :Lfr, :Les, :Dde, :Den])
        set_property_list(out_stream, row, "LAST", 
            [:P2, :P131, :P154, :P601, :P76, :P472, :P165, :Swikidatawiki, :Sdewiki], 
            [identity, identity, identity, fqs_string, fqs_string, fqs_string, fqs_string, fqs_string, fqs_string])
    end        
end